In [3]:
using QuantumOptics

In [4]:
using LinearAlgebra

In [2]:
1

1

In [8]:
## Example of LinearSolve
A = rand(4, 4)
b = rand(4)
sol = A\b

4-element Vector{Float64}:
 -0.09297450741739782
  0.3382379860223766
  0.28870526785024114
  0.8058447780589868

In [14]:
## Example Quantum Sytem to try the function out 
basis = FockBasis(2);
# creation and annihilation operators of the cavity
a = destroy(basis);
at = create(basis);
H = at*a + at*at*a*a;
# Jump operators and their rates to simulate coupling to a bath 
J = [a];
#Jt = dagger(a)

1-element Vector{Operator{FockBasis{Int64}, FockBasis{Int64}, SparseArrays.SparseMatrixCSC{ComplexF64, Int64}}}:
 Operator(dim=3x3)
  basis: Fock(cutoff=2)sparse([1, 2], [2, 3], ComplexF64[1.0 + 0.0im, 1.4142135623730951 + 0.0im], 3, 3)

In [25]:
α = coherentstate(basis,1+im);
ρα = α ⊗ dagger(α);
ραvec = vec(ρα.data);

In [26]:
L = Matrix(liouvillian(H, J;).data);
unitm = diagm(ones(size(L)[1]));

In [27]:
Mat = cat(L,unitm;dims=1);

In [28]:
ρss = vec(Matrix(steadystate.master(H,J)[2][1].data));

In [29]:
rhs = append!(ραvec - ρss*sum(ραvec),zeros(size(L)[1]));


In [24]:
Mat\rhs

9-element Vector{ComplexF64}:
                   0.0 - 0.0im
  -0.09098651177429772 + 0.0761082142514566im
  -0.04253177622684928 - 0.010632944056712322im
  -0.09098651177429777 - 0.07610821425145657im
   -0.5734440985830829 + 6.944925103571634e-17im
  -0.05907690693665959 + 0.02832454442168606im
  -0.04253177622684929 + 0.01063294405671232im
 -0.059076906936659575 - 0.0283245444216861im
   -0.1274320219073517 + 1.543316689682585e-17im

In [37]:
""" 

        drazin_apply() 
Calculates the vector resulting from the Drazin inverse being applied to another vector. 

# Arguments 
* `H`: Arbitrary operator specifying the Hamiltonian.
* `J`: Vector containing all jump operators which can be of any arbitrary
        operator type.
* `ρss ` : steady state of the system as a density matrix
* `α` : input state as a density matrix

"""
function drazin_apply(H, J, ρα, ρss = steadystate.master(H,J)[2][1])
        ## Constructing the matrix 

        # constructing the liouvillian from the Hamiltonian and the jump operators 
        L = Matrix(liouvillian(H, J;).data)

        # constructing unitmatrix to append to liouvillian
        unitm = diagm(ones(size(L)[1]))

        # constructing the final matrix consiting of the liouvillian and the unit matrix 
        Mat = cat(L,unitm;dims=1)

        ## Constructing the right hand side 

        # vectorizing the steady state
        ρssvec = vec(Matrix(ρss.data))

        # vectorizing the state that the drazin inverse is being applied to  
        αvec = vec(ρα.data)

        # constructing the right hand side of the linear system 
        rhs = append!(ραvec - ρssvec*sum(ραvec),zeros(size(L)[1]))

        ## returning the result 
        
        return Mat\rhs
end 

drazin_apply

In [39]:
drazin_apply(H,J,α ⊗ dagger(α))

9-element Vector{ComplexF64}:
                   0.0 - 0.0im
  -0.09098651177429772 + 0.0761082142514566im
  -0.04253177622684928 - 0.010632944056712322im
  -0.09098651177429777 - 0.07610821425145657im
   -0.5734440985830829 + 6.944925103571634e-17im
  -0.05907690693665959 + 0.02832454442168606im
  -0.04253177622684929 + 0.01063294405671232im
 -0.059076906936659575 - 0.0283245444216861im
   -0.1274320219073517 + 1.543316689682585e-17im